In [1]:
## Imports
import os
import sys
import random
from osgeo import gdal

import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

## Seeding 
seed = 2019
random.seed = seed
np.random.seed = seed
tf.seed = seed

C:\Users\jbrown\AppData\Local\conda\conda\envs\neural2\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jbrown\AppData\Local\conda\conda\envs\neural2\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jbrown\AppData\Local\conda\conda\envs\neural2\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jbrown\

In [2]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, mpath, batch_size=8, image_size=128):
        self.ids = ids
        self.path = path
        self.mpath = mpath
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
        
    def __load__(self, id_name):
        ## Path
        image_path = os.path.join(self.path, id_name)
        mask_path = os.path.join(self.mpath, id_name)

        image = cv2.imread(image_path, 1)
        image = cv2.resize(image, (self.image_size, self.image_size))
        mask = cv2.imread(mask_path, 1)
        mask = cv2.resize(mask, (self.image_size, self.image_size))
        
        return image, mask
    
    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        image = []
        mask  = []
        
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
            
        image = np.array(image)
        mask  = np.array(mask)
        
        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [5]:
image_size = 128
train_path = r"U:\Training_Data\Training_Images\Test\Unet_Train\T13UFQ_20180907_2"
epochs = 5
batch_size = 100
mpath = r'U:\Training_Data\Training_Images\Test\Raster_Masks\Train\T13UFQ_20180907'

## Training Ids
train_ids = next(os.walk(train_path))[2]

## Validation Data Size
val_data_size = 10

valid_ids = train_ids[:val_data_size]
train_ids = train_ids[val_data_size:]

In [6]:
train_ids

['1098_0.png',
 '1098_1098.png',
 '1098_1647.png',
 '1098_2196.png',
 '1098_2745.png',
 '1098_3294.png',
 '1098_3843.png',
 '1098_4392.png',
 '1098_4941.png',
 '1098_549.png',
 '1647_0.png',
 '1647_1098.png',
 '1647_1647.png',
 '1647_2196.png',
 '1647_2745.png',
 '1647_3294.png',
 '1647_3843.png',
 '1647_4392.png',
 '1647_4941.png',
 '1647_549.png',
 '2196_0.png',
 '2196_1098.png',
 '2196_1647.png',
 '2196_2196.png',
 '2196_2745.png',
 '2196_3294.png',
 '2196_3843.png',
 '2196_4392.png',
 '2196_4941.png',
 '2196_549.png',
 '2745_0.png',
 '2745_1098.png',
 '2745_1647.png',
 '2745_2196.png',
 '2745_2745.png',
 '2745_3294.png',
 '2745_3843.png',
 '2745_4392.png',
 '2745_4941.png',
 '2745_549.png',
 '3294_0.png',
 '3294_1098.png',
 '3294_1647.png',
 '3294_2196.png',
 '3294_2745.png',
 '3294_3294.png',
 '3294_3843.png',
 '3294_4392.png',
 '3294_4941.png',
 '3294_549.png',
 '3843_0.png',
 '3843_1098.png',
 '3843_1647.png',
 '3843_2196.png',
 '3843_2745.png',
 '3843_3294.png',
 '3843_3843.png

In [43]:
gen = DataGen(train_ids, train_path, mpath, batch_size=batch_size, image_size=image_size)
x, y = gen.__getitem__(0)
print(x.shape, y.shape)

(90, 128, 128, 3) (90, 128, 128, 3)


In [44]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [45]:
def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [48]:
model = UNet()
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 128, 128, 16) 448         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_51 (Conv2D)              (None, 128, 128, 16) 2320        conv2d_50[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_12 (MaxPooling2D) (None, 64, 64, 16)   0           conv2d_51[0][0]                  
__________________________________________________________________________________________________
conv2d_52 

In [49]:
train_gen = DataGen(train_ids, train_path, mpath, image_size=image_size, batch_size=batch_size)
valid_gen = DataGen(valid_ids, train_path, mpath, image_size=image_size, batch_size=batch_size)

train_steps = len(train_ids)//batch_size
valid_steps = len(valid_ids)//batch_size

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=train_steps, validation_steps=valid_steps, 
                    epochs=epochs)

Epoch 1/5


ValueError: A target array with shape (90, 128, 128, 3) was passed for an output of shape (None, 128, 128, 1) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.